# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import gmaps.datasets

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [56]:
#convert ..WeatherPy/cityweather.csv into df
cityweather = "WeatherPy/cityweather.csv"
cityweather_df = pd.read_csv(cityweather)

cityweather_df

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hasaki,35.7333,140.8333,59.29,62,100,11.86,JP,1635377839
1,1,Broome,42.2506,-75.8330,50.38,85,75,3.00,US,1635377839
2,2,Punta Arenas,-53.1500,-70.9167,51.91,50,0,13.80,CL,1635377713
3,3,Faanui,-16.4833,-151.7500,79.74,78,67,6.11,PF,1635377840
4,4,Valdivia,-39.8142,-73.2459,59.09,97,2,3.06,CL,1635377545
...,...,...,...,...,...,...,...,...,...,...
545,545,Ambunti,-4.2365,142.8448,82.29,87,64,0.96,PG,1635377993
546,546,Venice of the Philippines,4.6628,119.3928,83.53,71,56,5.35,PH,1635377993
547,547,Chicama,-7.8447,-79.1469,61.34,81,32,8.55,PE,1635377994
548,548,Kilindoni,-7.9139,39.6668,77.77,81,40,6.96,TZ,1635377994


In [57]:
#dropping unnamed column
cityweather_df.drop(['Unnamed: 0'], axis=1, inplace=True)
cityweather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hasaki,35.7333,140.8333,59.29,62,100,11.86,JP,1635377839
1,Broome,42.2506,-75.8330,50.38,85,75,3.00,US,1635377839
2,Punta Arenas,-53.1500,-70.9167,51.91,50,0,13.80,CL,1635377713
3,Faanui,-16.4833,-151.7500,79.74,78,67,6.11,PF,1635377840
4,Valdivia,-39.8142,-73.2459,59.09,97,2,3.06,CL,1635377545
...,...,...,...,...,...,...,...,...,...
545,Ambunti,-4.2365,142.8448,82.29,87,64,0.96,PG,1635377993
546,Venice of the Philippines,4.6628,119.3928,83.53,71,56,5.35,PH,1635377993
547,Chicama,-7.8447,-79.1469,61.34,81,32,8.55,PE,1635377994
548,Kilindoni,-7.9139,39.6668,77.77,81,40,6.96,TZ,1635377994


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [58]:
#verifing data types for heatmap. humidity was int lat and long are floats
#create values for lat, longs, and humidity
humidity = cityweather_df["Humidity"].astype(float)

geo_location = cityweather_df [["Latitude", "Longitude"]].astype(float)

#checking for NAN - none 

humidity.isnull().values.any()

False

In [61]:
#config for google heatmap
gmaps.configure(api_key = g_key)


#create heatmap layer
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(geo_location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
#show
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [62]:

#weather conditions max temp lower than 80 but higher than 70
#wind speed less than 10mph
#zero cloudiness
#drop values that do not contain all three
ideal_weather = cityweather_df.loc[(cityweather_df["Max Temp"] <= 80) & (cityweather_df["Max Temp"] >= 70) & (cityweather_df["Wind Speed"] <= 10) & (cityweather_df["Cloudiness"] == 0)].dropna()

ideal_weather


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,Salalah,17.0151,54.0924,73.49,56,0,4.61,OM,1635377858
243,Goya,-29.1400,-59.2626,79.97,40,0,9.31,AR,1635377904
269,Thatta,24.7475,67.9235,76.50,36,0,4.07,PK,1635377911
281,Khipro,25.8285,69.3772,76.30,26,0,5.59,PK,1635377915
447,Umluj,25.0213,37.2685,79.50,61,0,8.97,SA,1635377964
452,Canyon Lake,29.8752,-98.2625,74.46,37,0,3.00,US,1635377739
473,Bilma,18.6853,12.9164,79.54,17,0,3.53,NE,1635377972
475,Dzaoudzi,-12.7887,45.2699,76.89,88,0,8.05,YT,1635377972


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [63]:
####LIMIT ROWS FOR CALLING ON API

#store variable as hotel_df

hotel_df = ideal_weather[["City", "Country", "Latitude", "Longitude"]]
hotel_df

#adding hotel name to database
hotel_df = hotel_df.reindex(hotel_df.columns.tolist() + ["Hotel Name"], axis=1)


hotel_df = hotel_df.rename(columns ={"Latitude" : "Lat", "Longitude" : "Lng"}) 
hotel_df

,City,Country,Lat,Lng,Hotel Name
123,Salalah,OM,17.0151,54.0924,NaN
243,Goya,AR,-29.1400,-59.2626,NaN
269,Thatta,PK,24.7475,67.9235,NaN
281,Khipro,PK,25.8285,69.3772,NaN
447,Umluj,SA,25.0213,37.2685,NaN
452,Canyon Lake,US,29.8752,-98.2625,NaN
473,Bilma,NE,18.6853,12.9164,NaN
475,Dzaoudzi,YT,-12.7887,45.2699,NaN


In [64]:
# set parameters

params = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params)
print(response)

<Response [200]>


In [65]:
##forloop get lat, long, locations
#print log Beginning Data Retrieval 
print("Beginning Data Retrieval")
print("-----------------------------")

#for loop for data with try "processing Record" 
for index, row in hotel_df.iterrows():
    
    #get geo coordinates
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
   
    #using the search term and locations
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params)
    hotel_nm = request.json()
    try:
        
        #grab first hotel and place in df
        hotel_df.loc[index, "Hotel Name"] = hotel_nm["results"][0]["name"]
       
    except KeyError:
        print(f'Missing results, not found. Skipping...')
       
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")


Beginning Data Retrieval
-----------------------------
-----------------------------
Data Retrieval Complete
-----------------------------


In [66]:
#display df
hotel_df

,City,Country,Lat,Lng,Hotel Name
123,Salalah,OM,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
243,Goya,AR,-29.1400,-59.2626,Apart Costanera Goya
269,Thatta,PK,24.7475,67.9235,We Care Guest House & Shadi lawn
281,Khipro,PK,25.8285,69.3772,Mukesh Bhojraj Khatri Graphix Khipro
447,Umluj,SA,25.0213,37.2685,Moon light Furnished Units
452,Canyon Lake,US,29.8752,-98.2625,The Lodge At Turkey Cove
473,Bilma,NE,18.6853,12.9164,Expeditions Ténére voyages
475,Dzaoudzi,YT,-12.7887,45.2699,Le Rocher


In [67]:
#converting to floats and changing names due to errors
hotel_df["Lat"].astype(float)
hotel_df["Lng"].astype(float)


123    54.0924
243   -59.2626
269    67.9235
281    69.3772
447    37.2685
452   -98.2625
473    12.9164
475    45.2699
Name: Lng, dtype: float64

In [68]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [72]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))